In [2]:
import pandas as pd
import numpy as np

### Dependent Variables

In [3]:
Bank_Profitability = [['Proxies', 'Measurements',"Calculation Method"], 
             ["Bank Profitability" , "ROA","Net Income/Total Assets"], 
             ["Bank Profitability" , "ROE","Net Income/Total Equity"]]
           
headers = Bank_Profitability.pop(0)

df1 = pd.DataFrame(Bank_Profitability, columns = headers)
df1.index +=1
df1.index.name = "Dependent Variable"
df1

,Proxies,Measurements,Calculation Method
Dependent Variable,,,
1,Bank Profitability,ROA,Net Income/Total Assets
2,Bank Profitability,ROE,Net Income/Total Equity


### Independent Variables

In [4]:
Bank_Profitability_Determinants = [['Proxies', 'Measurements'], 
        ["Credit Risk","Loans Loss Provision/Loans, Net Charge-off/Total Loans"],
           ["Liquidity Risk", "Liquidity Assets/Assets"],
                        ["Bank Size", "ln of Total Assets"],
            ["Loans-to-Total Assets", "Loans/Assets"],
            ["Capital Ratio", "Equities/Assets"],
            ["Fixed Assets-to-Total Assets","Fixed Assets/Assets"],
            ["Off-balance sheet", "Total derivative contracts held for trading. Equity exchange contracts"],
            ["Off-balance sheet","Total derivative contracts held for trading. Commodity and other contracts"],
           
             ["Off-balance sheet","Total derivative contracts held for trading. Interest rate contracts"],
             ["Off-balance sheet","Total derivative contracts held for trading. Foreign exchange contracts"],
         ["Off-balance sheet","Current credit exposure across all off-balance sheet derivatives\
            contract covered by the risk based capital standards"],
            [" Real GDP","Millions of Chained 2009 Dollars"],
            ["Inflation","CPI,base year 2010 =100"],
            ["Slope of yeild curve", "the difference between 10-year Treasury bond rate and the 3-month Treasury bond rate"]                      ]
           
headers = Bank_Profitability_Determinants.pop(0)

df2 = pd.DataFrame(Bank_Profitability_Determinants, columns = headers)
df2.index +=1
df2.index.name = "Independent Variables"
df2

,Proxies,Measurements
Independent Variables,,
1,Credit Risk,"Loans Loss Provision/Loans, Net Charge-off/Tot..."
2,Liquidity Risk,Liquidity Assets/Assets
3,Bank Size,ln of Total Assets
4,Loans-to-Total Assets,Loans/Assets
5,Capital Ratio,Equities/Assets
6,Fixed Assets-to-Total Assets,Fixed Assets/Assets
7,Off-balance sheet,Total derivative contracts held for trading. E...
8,Off-balance sheet,Total derivative contracts held for trading. C...
9,Off-balance sheet,Total derivative contracts held for trading. I...


#### Quarterly Data over period Q1 2000 to Q4 2015

##### Load Real GDP and Inflation data
Source: Fred, Federal Reserve Bank of St.Louis

In [5]:
economic_metric = pd.read_csv("economic_metric.csv")
economic_metric.tail()

,Quarter,DATE,Real GDP ($M),"Inflation (CPI, base year 2010 =100)"
59,Q4 2014,10/1/2014,16186736,108.3
60,Q1 2015,1/1/2015,16268979,107.7
61,Q2 2015,4/1/2015,16374177,109.0
62,Q3 2015,7/1/2015,16454877,109.3
63,Q4 2015,10/1/2015,16490681,108.8


##### Load 10-year and 3-month Tresury constant maturity rate to calculate slope of yeild curve
Source: Fred, Federal Reserve Bank of St.Louis

In [6]:
slope = pd.read_csv("DGS10.csv")
slope.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4174 entries, 0 to 4173
Data columns (total 3 columns):
DATE            4174 non-null object
10-year rate    4147 non-null object
3-month rate    4147 non-null object
dtypes: object(3)
memory usage: 97.9+ KB


Remove rows with "." and blank columns

In [7]:
slope = slope[slope["10-year rate"] != '.']
slope = slope[slope["3-month rate"] != '.']
slope = slope.dropna()

Calculate slope of yeild curve

In [8]:
slope["Slope of yeild curve"]= slope["10-year rate"].astype("float64") - slope["3-month rate"].astype("float64")

Manipulate slope dataframe to get only quarterly slope as the dataframe currently contains daily data.

Step 1: split DATA into day, month, year

In [9]:
month = []
day = []
year =[]
for i in slope["DATE"]:
    m = i.split("/")[0]
    month.append(m)
    d = i.split("/")[1]
    day.append(d)
    y = i.split("/")[2]
    year.append(y)


In [10]:
slope["month"]= month
slope["day"]= day
slope["year"]= year

In [11]:
slope[:4]

,DATE,10-year rate,3-month rate,Slope of yeild curve,month,day,year
0,1/3/2000,6.58,5.48,1.10,1,3,2000
1,1/4/2000,6.49,5.43,1.06,1,4,2000
2,1/5/2000,6.62,5.44,1.18,1,5,2000
3,1/6/2000,6.57,5.41,1.16,1,6,2000


Step 2: Remove rows which do not contain March, June, September, December

In [12]:
slope = slope[-slope["month"].isin(["3", "6", "9","12"])== False]


In [13]:
slope["month"] = slope["month"].astype("int64")
slope["day"] = slope["day"].astype("int64")
slope["year"] = slope["year"].astype("int64")

Step 3: Get only data of the last day of the months

Step 3.1: Convert columns["year","month","day"] into indexes

In [14]:
new = slope.set_index(["year","month","day"])
new[:2]

DATE 10-year rate 3-month rate  Slope of yeild curve
year month day                                                          
2000 3     1    3/1/2000         6.39         5.76                  0.63
           2    3/2/2000          6.4         5.76                  0.64

Step 3.2: Create a loop to get rows containing 'Slope of yeild curve' of last day of the months and then group them into a new dataframe

In [15]:
data = []
for y in slope["year"].unique():
    for m in slope["month"].unique():
        item = new.ix[y,m][-1:]
        data.append(item)

In [16]:
data[:2]

[          DATE 10-year rate 3-month rate  Slope of yeild curve
 day                                                           
 31   3/31/2000         6.03         5.88                  0.15,
           DATE 10-year rate 3-month rate  Slope of yeild curve
 day                                                           
 30   6/30/2000         6.03         5.88                  0.15]

In [17]:
slope1 = pd.DataFrame()

In [18]:
slope1 = slope1.append(data)
slope1[:3]

,DATE,10-year rate,3-month rate,Slope of yeild curve
day,,,,
31,3/31/2000,6.03,5.88,0.15
30,6/30/2000,6.03,5.88,0.15
29,9/29/2000,5.8,6.23,-0.43


In [19]:
slope2 = slope1.reset_index()
slope2[:4]

,day,DATE,10-year rate,3-month rate,Slope of yeild curve
0,31,3/31/2000,6.03,5.88,0.15
1,30,6/30/2000,6.03,5.88,0.15
2,29,9/29/2000,5.8,6.23,-0.43
3,29,12/29/2000,5.12,5.89,-0.77


Step 4: Append a column[Quarter] to specify the quarters of the slope of yeild curve

In [20]:
slope2['DATE'][0].split('/')[2]

'2000'

In [21]:
i = "Q"+ "1" +" " +slope2['DATE'][0].split("/")[2]
i

'Q1 2000'

In [22]:
quarter = []
for i in slope1["DATE"]:
    if i.split('/')[0] == "3":
        i = "Q"+ "1" +" " +i.split("/")[2]
        quarter.append(i)
    elif i.split('/')[0] == "6":
        i = "Q"+ "2" +" " +i.split("/")[2]
        quarter.append(i)
    elif i.split('/')[0] == "9":
        i = "Q"+ "3" +" " +i.split("/")[2]
        quarter.append(i)
    else:
        i = "Q"+ "4" +" " +i.split("/")[2]
        quarter.append(i)

len(quarter)

64

In [23]:
slope2["Quarter"] = quarter
slope2[:2]


,day,DATE,10-year rate,3-month rate,Slope of yeild curve,Quarter
0,31,3/31/2000,6.03,5.88,0.15,Q1 2000
1,30,6/30/2000,6.03,5.88,0.15,Q2 2000


In [24]:
slope3 = slope2.drop(["day","10-year rate","3-month rate","DATE"],axis=1)
slope3[:2]

,Slope of yeild curve,Quarter
0,0.15,Q1 2000
1,0.15,Q2 2000


Create a dataframe composing of macro variables

In [25]:
macro_data = pd.merge(slope3,economic_metric)
macro_data[:2]


,Slope of yeild curve,Quarter,DATE,Real GDP ($M),"Inflation (CPI, base year 2010 =100)"
0,0.15,Q1 2000,1/1/2000,12359095,77.9
1,0.15,Q2 2000,4/1/2000,12592530,78.8


Rearrange order of columns

In [26]:
macro_data = macro_data[["Quarter","Real GDP ($M)",
                         "Inflation (CPI, base year 2010 =100)",
                        "Slope of yeild curve"]]

macro_data[:3]

,Quarter,Real GDP ($M),"Inflation (CPI, base year 2010 =100)",Slope of yeild curve
0,Q1 2000,12359095,77.9,0.15
1,Q2 2000,12592530,78.8,0.15
2,Q3 2000,12607676,79.4,-0.43


Import Net Income Items

In [27]:
net_income_item =  pd.read_csv("net income1.csv")

C:\Users\thanh\AppData\Local\Enthought\Canopy\User\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
net_income_item.columns = ["Loan Loss Provision","Net Income","Total Loan Recoveries",
                           "Total Loan Charge-offs",\
               "Bank ID","Bank Name","City","State","Zip Code",
                           "Date established","Reporting Date"]

In [29]:
net_income_item[:4]


,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Bank Name,City,State,Zip Code,Date established,Reporting Date
0,27.0,221.0,26.0,24.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,20000331
1,78.0,453.0,42.0,77.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,20000630
2,129.0,697.0,58.0,124.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,20000930
3,180.0,908.0,67.0,205.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,20001231


Some banks changed their names. Thus number of Bank ID # number of Bank Name

In [30]:
net_income_item["Bank Name"].nunique()

12036

In [31]:
len(net_income_item["Bank Name"].unique())

12037

In [32]:
net_income_item["Bank ID"].nunique()

11731

In [33]:
len(net_income_item["Bank ID"].unique())

11731

In [34]:
net_income_item.ix[net_income_item["Bank ID"] == 2938][35:-1]
# This bank changed its name since 03/31/2009

,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Bank Name,City,State,Zip Code,Date established,Reporting Date
1459,4759.0,1200.0,510.0,2253.0,2938,SECURITY B&TC OF ALBANY,ALBANY,GA,31701,19700831.0,20081231
1460,3877.0,-994.0,39.0,4527.0,2938,SB&T BK,ALBANY,GA,31701,19700831.0,20090331
1461,6510.0,-1902.0,254.0,6223.0,2938,SB&T BK,ALBANY,GA,31701,19700831.0,20090630
1462,14770.0,-5080.0,293.0,14623.0,2938,SB&T BK,ALBANY,GA,31701,19700831.0,20090930
1463,20047.0,-6509.0,508.0,21204.0,2938,SB&T BK,ALBANY,GA,31701,19700831.0,20091231


Import Balance sheet items

In [35]:
balance_sheet_item =  pd.read_csv("balance sheet.csv")

In [36]:
balance_sheet_item.columns = ["Cash", "Available-for-sale securities",
                              "Total Loan, net of unearned income","Fixed Assets",
                              "Total Assets",\
               "Total Equity","Bank ID","Bank Name","Reporting Date"]

balance_sheet_item[:3]

,Cash,Available-for-sale securities,"Total Loan, net of unearned income",Fixed Assets,Total Assets,Total Equity,Bank ID,Bank Name,Reporting Date
0,1436.0,198.0,36620.0,614.0,64088.0,10709.0,37,BANK OF HANCOCK CTY,20000331
1,2014.0,198.0,38458.0,597.0,62982.0,10845.0,37,BANK OF HANCOCK CTY,20000630
2,1386.0,197.0,41145.0,589.0,63862.0,11089.0,37,BANK OF HANCOCK CTY,20000930


In [37]:
len(balance_sheet_item["Bank ID"].unique())

11731

Import Off-balance sheet items

In [248]:
obs_item =  pd.read_csv("obs.csv")

In [249]:
obs_item.columns = ["Total derivative contracts held for trading. Equity exchange contracts", "Total derivative contracts \
held for trading. Commodity and other contracts","Current credit exposure across all off-balance sheet derivatives\
            contract covered by the risk based capital standards","Total derivative contracts\
            held for trading. Interest rate contracts","Total derivative contracts held for trading. Foreign exchange contracts",\
               "Bank ID","Bank Name","Reporting Date"]

obs_item[:3]

,Total derivative contracts held for trading. Equity exchange contracts,Total derivative contracts held for trading. Commodity and other contracts,Current credit exposure across all off-balance sheet derivatives contract covered by the risk based capital standards,Total derivative contracts held for trading. Interest rate contracts,Total derivative contracts held for trading. Foreign exchange contracts,Bank ID,Bank Name,Reporting Date
0,0.0,0.0,0.0,0.0,0.0,37,BANK OF HANCOCK CTY,20000331
1,0.0,0.0,0.0,0.0,0.0,37,BANK OF HANCOCK CTY,20000630
2,0.0,0.0,0.0,0.0,0.0,37,BANK OF HANCOCK CTY,20000930


In [250]:
len(obs_item["Bank ID"].unique())

11731

In [251]:
df = pd.merge(net_income_item,balance_sheet_item)

In [252]:
df = pd.merge(df,obs_item)

In [253]:
df[:3]

,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Bank Name,City,State,Zip Code,Date established,...,Available-for-sale securities,"Total Loan, net of unearned income",Fixed Assets,Total Assets,Total Equity,Total derivative contracts held for trading. Equity exchange contracts,Total derivative contracts held for trading. Commodity and other contracts,Current credit exposure across all off-balance sheet derivatives contract covered by the risk based capital standards,Total derivative contracts held for trading. Interest rate contracts,Total derivative contracts held for trading. Foreign exchange contracts
0,27.0,221.0,26.0,24.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,...,198.0,36620.0,614.0,64088.0,10709.0,0.0,0.0,0.0,0.0,0.0
1,78.0,453.0,42.0,77.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,...,198.0,38458.0,597.0,62982.0,10845.0,0.0,0.0,0.0,0.0,0.0
2,129.0,697.0,58.0,124.0,37,BANK OF HANCOCK CTY,SPARTA,GA,31087,19040901.0,...,197.0,41145.0,589.0,63862.0,11089.0,0.0,0.0,0.0,0.0,0.0


Add to df a column[Quarter] to specify the quarters 

In [254]:
df["Reporting Date"] = df["Reporting Date"].astype('str')

In [255]:
quarter1 = []
for i in df["Reporting Date"]:
    if i[4:6] == "03":
        i = "Q"+ "1" +" " +i[0:4]
        quarter1.append(i)
    elif i[4:6] == "06":
        i = "Q"+ "2" +" " +i[0:4]
        quarter1.append(i)
    elif i[4:6] == "09":
        i = "Q"+ "3" +" " +i[0:4]
        quarter1.append(i)
    else:
        i = "Q"+ "4" +" " +i[0:4]
        quarter1.append(i)

len(quarter1)

511279

In [256]:
df["Quarter"]= quarter1

In [257]:
df1 = df.drop(['Bank Name','City','State','Zip Code','Date established'],axis =1)

In [258]:
df1.sample()

,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Reporting Date,Cash,Available-for-sale securities,"Total Loan, net of unearned income",Fixed Assets,Total Assets,Total Equity,Total derivative contracts held for trading. Equity exchange contracts,Total derivative contracts held for trading. Commodity and other contracts,Current credit exposure across all off-balance sheet derivatives contract covered by the risk based capital standards,Total derivative contracts held for trading. Interest rate contracts,Total derivative contracts held for trading. Foreign exchange contracts,Quarter
220818,360.0,2501.0,112.0,124.0,573335,20061231,4290.0,43682.0,112690.0,2563.0,173663.0,14813.0,0.0,0.0,0.0,0.0,0.0,Q4 2006


Combine macro variables and banks related variables into 1 data frame

In [259]:
df2 = pd.merge(df1, macro_data, how ='outer')

In [260]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 511279 entries, 0 to 511278
Data columns (total 21 columns):
Loan Loss Provision                                                                                                                 441710 non-null float64
Net Income                                                                                                                          441982 non-null float64
Total Loan Recoveries                                                                                                               441654 non-null float64
Total Loan Charge-offs                                                                                                              438611 non-null float64
Bank ID                                                                                                                             511279 non-null int64
Reporting Date                                                                                                 

In [261]:
df2["Net Charge-off"] = df2["Total Loan Charge-offs"] - df2["Total Loan Recoveries"]

df2["ROA"] =df2["Net Income"]/df2["Total Assets"]

df2["ROE"] = df2["Net Income"]/df2["Total Equity"]

df2["Loan Loss Provision %"] = df2["Loan Loss Provision"]/df2["Total Loan, \
net of unearned income"]

df2["Net Charge Off-to-Loans"] = df2["Net Charge-off"]/df2["Total Loan, net of unearned income"]

df2["Liquidity"] = (df2["Cash"]+df2["Available-for-sale securities"])/df2["Total Assets"]

df2["Bank Size"] = np.log(df2["Total Assets"].astype("float64"))

df2["Loan-to-Assets"] = df2["Total Loan, net of unearned income"]/df2["Total Assets"]

df2["Leverage"] = df2["Total Equity"]/df2["Total Assets"]

df2["Fixed Assets-to-Assets"] = df2["Fixed Assets"]/df2["Total Assets"]


In [262]:
pd.isnull(df2).any(1).nonzero()[0]

array([    11,     29,     31, ..., 511276, 511277, 511278], dtype=int64)

Drop rows where ROA and ROE are NaN

In [263]:
df3 = df2[df2['ROA'].notnull() & df2['ROE'].notnull()]

In [264]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441900 entries, 0 to 511270
Data columns (total 31 columns):
Loan Loss Provision                                                                                                                 441638 non-null float64
Net Income                                                                                                                          441900 non-null float64
Total Loan Recoveries                                                                                                               441575 non-null float64
Total Loan Charge-offs                                                                                                              438609 non-null float64
Bank ID                                                                                                                             441900 non-null int64
Reporting Date                                                                                                 

In [265]:
df3[:3]

,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Reporting Date,Cash,Available-for-sale securities,"Total Loan, net of unearned income",Fixed Assets,...,Net Charge-off,ROA,ROE,Loan Loss Provision %,Net Charge Off-to-Loans,Liquidity,Bank Size,Loan-to-Assets,Leverage,Fixed Assets-to-Assets
0,27.0,221.0,26.0,24.0,37,20000331,1436.0,198.0,36620.0,614.0,...,-2.0,0.003448,0.020637,0.000737,-0.000055,0.025496,11.068012,0.571402,0.167098,0.009581
1,0.0,69.0,4.0,2.0,242,20000331,774.0,9803.0,11710.0,324.0,...,-2.0,0.003021,0.021725,0.000000,-0.000171,0.463111,10.036225,0.512719,0.139060,0.014186
2,0.0,166.0,0.0,0.0,279,20000331,1351.0,28282.0,20456.0,603.0,...,0.0,0.001816,0.012053,0.000000,0.000000,0.324234,11.422935,0.223822,0.150699,0.006598


Add a new column specifing only the quarters, then use that column to create dummy variables

In [266]:
quarter2 = []
for i in df3["Quarter"]:
    quarter2.append(i[1])

# create funtion to return unique values of a list.    
def unique(llist):
    unique = []
    for i in llist:
        if i not in unique:
            unique.append(i)
    return unique

unique(quarter2)

['1', '2', '3', '4']

In [267]:
df3['Quarter ID'] = quarter2

C:\Users\thanh\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [268]:
df3_quarterID = pd.get_dummies(df3['Quarter ID'])

In [269]:
df4 = pd.concat([df3, df3_quarterID],axis =1 )

Drop column '1' as it's redundant

In [270]:
df4 = df4.drop(['1'],axis =1)

In [271]:
df4.sample()

,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Reporting Date,Cash,Available-for-sale securities,"Total Loan, net of unearned income",Fixed Assets,...,Net Charge Off-to-Loans,Liquidity,Bank Size,Loan-to-Assets,Leverage,Fixed Assets-to-Assets,Quarter ID,2,3,4
240666,228.0,5023.0,0.0,0.0,759045,20061231,4336.0,32792.0,244804.0,7337.0,...,0.0,0.124278,12.607359,0.81943,0.102661,0.024559,4,0.0,0.0,1.0


Change name of column '2', '3', '4' to Q2, Q3, Q4 respectively

In [272]:
df4.rename(columns={'2': 'Q2', '3': 'Q3', '4': 'Q4'}, inplace=True)

In [273]:
df4.sample()

,Loan Loss Provision,Net Income,Total Loan Recoveries,Total Loan Charge-offs,Bank ID,Reporting Date,Cash,Available-for-sale securities,"Total Loan, net of unearned income",Fixed Assets,...,Net Charge Off-to-Loans,Liquidity,Bank Size,Loan-to-Assets,Leverage,Fixed Assets-to-Assets,Quarter ID,Q2,Q3,Q4
255670,45.0,740.0,4.0,54.0,528241,20070630,3157.0,0.0,95880.0,1407.0,...,0.000521,0.024946,11.748409,0.757633,0.113985,0.011118,2,1.0,0.0,0.0


In [292]:
df5 = df4.drop(['Loan Loss Provision', 'Net Income', 'Total Loan Recoveries',
       'Total Loan Charge-offs','Cash',
       'Available-for-sale securities', 'Total Loan, net of unearned income',
       'Fixed Assets', u'Total Assets', 'Total Equity','Quarter ID'], axis =1 )

In [293]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441900 entries, 0 to 511270
Data columns (total 24 columns):
Bank ID                                                                                                                             441900 non-null int64
Reporting Date                                                                                                                      441900 non-null object
Total derivative contracts held for trading. Equity exchange contracts                                                              438609 non-null float64
Total derivative contracts held for trading. Commodity and other contracts                                                          438609 non-null float64
Current credit exposure across all off-balance sheet derivatives            contract covered by the risk based capital standards    436071 non-null float64
Total derivative contracts            held for trading. Interest rate contracts                                 

Create dummy variables for the following variables: 'Loan Loss Provision %','Net Charge Off-to-Loans', 'Liquidity', Off-balance sheet variables

- Create a function to generate a list composing of 2 unique values: 0, 1
- 0 means that the corresponding value of the variable is an NaN.
- 1 means that the corresponding value of the variable is not an NaN.

In [295]:
def dummy_variable(a):
    dummy = []
    for i in a:
        if np.isnan(i) == True:
            dummy.append(0)
        else:
            dummy.append(1)
    return dummy
        

In [296]:
df5['Dummy_Loan Loss Provision %'] = dummy_variable(df5['Loan Loss Provision %'])

df5['Dummy_Net Charge Off-to-Loans'] = dummy_variable(df5['Net Charge Off-to-Loans'])

df5['Dummy_Liquidity'] = dummy_variable(df5['Liquidity'])

df5['Dummy_OBS_Equity'] = dummy_variable(df5['Total derivative contracts held for trading. Equity exchange contracts'])

df5['Dummy_OBS_Commodity&Other'] = dummy_variable(df5['Total derivative contracts held for trading. Commodity and other contracts'])

df5['Dummy_OBS_Interest'] = dummy_variable(df5['Total derivative contracts            held for trading. Interest rate contracts'])

df5['Dummy_OBS_FX'] = dummy_variable(df5['Total derivative contracts held for trading. Foreign exchange contracts'])

df5['Dummy_OBS_Credit_exposure'] = dummy_variable(df5['Current credit exposure across all off-balance sheet derivatives            contract covered by the risk based capital standards'])

In [299]:
len(df5.ix[df5['Total derivative contracts held for trading. Equity exchange contracts'].isnull()])

3291

In [300]:
len(df5.ix[df5['Liquidity'].isnull()])

2505

Fill NaN with 0

In [301]:
df6 = df5.fillna(0)

In [306]:
df6.columns

Index([u'Bank ID', u'Reporting Date',
       u'Total derivative contracts held for trading. Equity exchange contracts',
       u'Total derivative contracts held for trading. Commodity and other contracts',
       u'Current credit exposure across all off-balance sheet derivatives            contract covered by the risk based capital standards',
       u'Total derivative contracts            held for trading. Interest rate contracts',
       u'Total derivative contracts held for trading. Foreign exchange contracts',
       u'Quarter', u'Real GDP ($M)', u'Inflation (CPI, base year 2010 =100)',
       u'Slope of yeild curve', u'Net Charge-off', u'ROA', u'ROE',
       u'Loan Loss Provision %', u'Net Charge Off-to-Loans', u'Liquidity',
       u'Bank Size', u'Loan-to-Assets', u'Leverage', u'Fixed Assets-to-Assets',
       u'Q2', u'Q3', u'Q4', u'Dummy_Loan Loss Provision %',
       u'Dummy_Net Charge Off-to-Loans', u'Dummy_Liquidity',
       u'Dummy_OBS_Equity', u'Dummy_OBS_Commodity&Other',
 

In [307]:
df7 = df6.drop('Reporting Date', axis = 1)

In [450]:
base_df = df7[['Quarter','Bank ID','ROA','ROE','Net Charge-off',
            'Loan Loss Provision %', 'Dummy_Loan Loss Provision %',
             'Net Charge Off-to-Loans','Dummy_Net Charge Off-to-Loans',
            'Liquidity', 'Dummy_Liquidity',
             'Bank Size',
             'Loan-to-Assets',
             'Leverage',
             'Fixed Assets-to-Assets',
'Total derivative contracts held for trading. Equity exchange contracts','Dummy_OBS_Equity',
'Total derivative contracts held for trading. Commodity and other contracts','Dummy_OBS_Commodity&Other',
'Total derivative contracts            held for trading. Interest rate contracts','Dummy_OBS_Interest',
'Total derivative contracts held for trading. Foreign exchange contracts','Dummy_OBS_FX',
'Current credit exposure across all off-balance sheet derivatives            contract covered by the risk based capital standards',
             'Dummy_OBS_Credit_exposure',
             'Real GDP ($M)','Inflation (CPI, base year 2010 =100)','Slope of yeild curve',
             'Q2', 'Q3', 'Q4'
             
            ]]

In [608]:
base_df.to_csv('base_df.csv')

In [452]:
base_df[-9:]

,Quarter,Bank ID,ROA,ROE,Net Charge-off,Loan Loss Provision %,Dummy_Loan Loss Provision %,Net Charge Off-to-Loans,Dummy_Net Charge Off-to-Loans,Liquidity,...,Total derivative contracts held for trading. Foreign exchange contracts,Dummy_OBS_FX,Current credit exposure across all off-balance sheet derivatives contract covered by the risk based capital standards,Dummy_OBS_Credit_exposure,Real GDP ($M),"Inflation (CPI, base year 2010 =100)",Slope of yeild curve,Q2,Q3,Q4
510895,Q4 2015,3385968,0.006751,0.009222,0.0,-0.005382,1,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511092,Q4 2015,3601097,11.907692,-0.892219,0.0,0.000000,0,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511228,Q4 2015,3856066,0.026281,0.267250,0.0,0.000988,1,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511254,Q4 2015,4165765,-0.280417,-0.343039,0.0,0.000000,0,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511257,Q4 2015,4196291,0.032581,0.057506,0.0,0.027690,1,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511267,Q4 2015,4388788,0.006383,0.006504,0.0,0.000000,0,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511268,Q4 2015,4388797,0.005909,0.005909,0.0,0.000000,0,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511269,Q4 2015,4392806,0.021177,0.116290,0.0,0.002640,1,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0
511270,Q4 2015,4422730,0.007058,0.171535,0.0,0.002608,1,0.0,0,0.0,...,0.0,0,0.0,0,16490681,108.8,2.11,0.0,0.0,1.0


Slice Q4 2015 data

In [454]:
Q4_2015 = base_df.ix[base_df['Quarter']=="Q4 2015"]

Q4 2015 has 38 entries

Slice Q3 2015

In [572]:
Q3_2015 = base_df.ix[base_df['Quarter']=="Q3 2015"]

"""Q3_2015 = Q3_2015.drop(['ROE','Net Charge-off','Quarter','Bank ID'], axis=1)
Q3_2015.rename(columns={'ROA': 'ROA Q3/2015', 
                        'Loan Loss Provision %': 'Loan Loss Provision % Q3/2015', 
    'Dummy_Loan Loss Provision %': 'Dummy_Loan Loss Provision % Q3/2015',
    'Net Charge Off-to-Loans':'Net Charge Off-to-Loans Q3/2015',
    'Dummy_Net Charge Off-to-Loans': 'Dummy_Net Charge Off-to-Loans Q3/2015',
    'Liquidity': 'Liquidity Q3/3015',
    'Dummy_Liquidity': 'Dummy_Liquidity Q3/2015',
    'Bank Size': 'Bank Size Q3/2015' ,
             'Loan-to-Assets': 'Loan-to-Assets Q3/2015',
             'Leverage': 'Leverage Q3/2015' ,
             'Fixed Assets-to-Assets':'Fixed Assets-to-Assets Q3/2015' ,
'Total derivative contracts held for trading. Equity exchange contracts': 'OBS_Equity Q3/2015',
                        'Dummy_OBS_Equity': 'Dummy_OBS_Equity Q3/2015',
'Total derivative contracts held for trading. Commodity and other contracts': 'OBS_Commodity&Other Q3/2015',
            'Dummy_OBS_Commodity&Other': 'Dummy_OBS_Commodity&Other Q3/2015',
'Total derivative contracts            held for trading. Interest rate contracts':'OBS_Interest Q3/2015',
        'Dummy_OBS_Interest':'dummy_OBS_Interest Q3/2015',
'Total derivative contracts held for trading. Foreign exchange contracts': 'OBS_FX Q3/2015',
        'Dummy_OBS_FX': 'Dummy_OBS_FX Q3/2015',
'Current credit exposure across all off-balance sheet derivatives            contract covered \
by the risk based capital standards':'OBS Credit Exposure Q3/2015',
             'Dummy_OBS_Credit_exposure': 'Dummy_OBS_Credit_exposure Q3/2015',
             'Real GDP ($M)': 'Real GDP ($M) Q3/2015',
            'Inflation (CPI, base year 2010 =100)':'Inflation Q3/2015',
            'Slope of yeild curve': 'Slope of yeild curve Q3/2015',
             'Q2' :'Q2 Q3/2015', 
            'Q3': 'Q3 Q3/2015', 
            'Q4':'Q4 Q3/2015'
                       }, inplace=True)"""

"Q3_2015 = Q3_2015.drop(['ROE','Net Charge-off','Quarter','Bank ID'], axis=1)\nQ3_2015.rename(columns={'ROA': 'ROA Q3/2015', \n                        'Loan Loss Provision %': 'Loan Loss Provision % Q3/2015', \n    'Dummy_Loan Loss Provision %': 'Dummy_Loan Loss Provision % Q3/2015',\n    'Net Charge Off-to-Loans':'Net Charge Off-to-Loans Q3/2015',\n    'Dummy_Net Charge Off-to-Loans': 'Dummy_Net Charge Off-to-Loans Q3/2015',\n    'Liquidity': 'Liquidity Q3/3015',\n    'Dummy_Liquidity': 'Dummy_Liquidity Q3/2015',\n    'Bank Size': 'Bank Size Q3/2015' ,\n             'Loan-to-Assets': 'Loan-to-Assets Q3/2015',\n             'Leverage': 'Leverage Q3/2015' ,\n             'Fixed Assets-to-Assets':'Fixed Assets-to-Assets Q3/2015' ,\n'Total derivative contracts held for trading. Equity exchange contracts': 'OBS_Equity Q3/2015',\n                        'Dummy_OBS_Equity': 'Dummy_OBS_Equity Q3/2015',\n'Total derivative contracts held for trading. Commodity and other contracts': 'OBS_Commodi

In [573]:
Q3_2015 = Q3_2015['ROA']


In [574]:
Q3_2015.head()

498328    0.019933
499042   -0.154141
499926    0.013786
500904    0.003439
501023    0.003605
Name: ROA, dtype: float64

In [576]:
Q3_2015 = Q3_2015.reset_index()
Q3_2015 = Q3_2015.drop(['index'],axis=1)
Q3_2015 = Q3_2015.rename(columns={'ROA': 'ROA Q3/2015'})
Q3_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 1 columns):
ROA Q3/2015    31 non-null float64
dtypes: float64(1)
memory usage: 320.0 bytes


Q3_2015 has 31 entries

Slice Q2 2015

In [462]:
Q2_2015 = base_df.ix[base_df['Quarter']=="Q2 2015"]
Q2_2015 = Q2_2015.drop(['ROE','Net Charge-off','Quarter','Bank ID'], axis=1)
Q2_2015.rename(columns={'ROA': 'ROA Q2/2015', 
                        'Loan Loss Provision %': 'Loan Loss Provision % Q2/2015', 
    'Dummy_Loan Loss Provision %': 'Dummy_Loan Loss Provision % Q2/2015',
    'Net Charge Off-to-Loans':'Net Charge Off-to-Loans Q2/2015',
    'Dummy_Net Charge Off-to-Loans': 'Dummy_Net Charge Off-to-Loans Q2/2015',
    'Liquidity': 'Liquidity Q2/2015',
    'Dummy_Liquidity': 'Dummy_Liquidity Q2/2015',
    'Bank Size': 'Bank Size Q2/2015' ,
             'Loan-to-Assets': 'Loan-to-Assets Q2/2015',
             'Leverage': 'Leverage Q2/2015' ,
             'Fixed Assets-to-Assets':'Fixed Assets-to-Assets Q2/2015' ,
'Total derivative contracts held for trading. Equity exchange contracts': 'OBS_Equity Q2/2015',
                        'Dummy_OBS_Equity': 'Dummy_OBS_Equity Q2/2015',
'Total derivative contracts held for trading. Commodity and other contracts': 'OBS_Commodity&Other Q2/2015',
            'Dummy_OBS_Commodity&Other': 'Dummy_OBS_Commodity&Other Q2/2015',
'Total derivative contracts            held for trading. Interest rate contracts':'OBS_Interest Q2/2015',
        'Dummy_OBS_Interest':'dummy_OBS_Interest Q2/2015',
'Total derivative contracts held for trading. Foreign exchange contracts': 'OBS_FX Q2/2015',
        'Dummy_OBS_FX': 'Dummy_OBS_FX Q2/2015',
'Current credit exposure across all off-balance sheet derivatives            contract covered \
by the risk based capital standards':'OBS Credit Exposure Q2/2015',
             'Dummy_OBS_Credit_exposure': 'Dummy_OBS_Credit_exposure Q2/2015',
             'Real GDP ($M)': 'Real GDP ($M) Q2/2015',
            'Inflation (CPI, base year 2010 =100)':'Inflation Q2/2015',
            'Slope of yeild curve': 'Slope of yeild curve Q2/2015',
             'Q2' :'Q2 Q2/2015', 
            'Q3': 'Q3 Q2/2015', 
            'Q4':'Q4 Q2/2015'
                       }, inplace=True)

In [463]:
Q2_2015 = Q2_2015.reset_index()
Q2_2015 = Q2_2015.drop(['index'],axis=1)

Q2_2015 has 31 entries

Slice Q1 2015

In [464]:
Q1_2015 = base_df.ix[base_df['Quarter']=="Q1 2015"]
Q1_2015 = Q1_2015.drop(['ROE','Net Charge-off','Quarter','Bank ID'], axis=1)
Q1_2015.rename(columns={'ROA': 'ROA Q1/2015', 
                        'Loan Loss Provision %': 'Loan Loss Provision % Q1/2015', 
    'Dummy_Loan Loss Provision %': 'Dummy_Loan Loss Provision % Q1/2015',
    'Net Charge Off-to-Loans':'Net Charge Off-to-Loans Q1/2015',
    'Dummy_Net Charge Off-to-Loans': 'Dummy_Net Charge Off-to-Loans Q1/2015',
    'Liquidity': 'Liquidity Q1/2015',
    'Dummy_Liquidity': 'Dummy_Liquidity Q1/2015',
    'Bank Size': 'Bank Size Q1/2015' ,
             'Loan-to-Assets': 'Loan-to-Assets Q1/2015',
             'Leverage': 'Leverage Q1/2015' ,
             'Fixed Assets-to-Assets':'Fixed Assets-to-Assets Q1/2015' ,
'Total derivative contracts held for trading. Equity exchange contracts': 'OBS_Equity Q1/2015',
                        'Dummy_OBS_Equity': 'Dummy_OBS_Equity Q1/2015',
'Total derivative contracts held for trading. Commodity and other contracts': 'OBS_Commodity&Other Q1/2015',
            'Dummy_OBS_Commodity&Other': 'Dummy_OBS_Commodity&Other Q1/2015',
'Total derivative contracts            held for trading. Interest rate contracts':'OBS_Interest Q1/2015',
        'Dummy_OBS_Interest':'dummy_OBS_Interest Q1/2015',
'Total derivative contracts held for trading. Foreign exchange contracts': 'OBS_FX Q1/2015',
        'Dummy_OBS_FX': 'Dummy_OBS_FX Q1/2015',
'Current credit exposure across all off-balance sheet derivatives            contract covered by \
the risk based capital standards':'OBS Credit Exposure Q1/2015',
             'Dummy_OBS_Credit_exposure': 'Dummy_OBS_Credit_exposure Q1/2015',
             'Real GDP ($M)': 'Real GDP ($M) Q1/2015',
            'Inflation (CPI, base year 2010 =100)':'Inflation Q1/2015',
            'Slope of yeild curve': 'Slope of yeild curve Q1/2015',
             'Q2' :'Q2 Q1/2015', 
            'Q3': 'Q3 Q1/2015', 
            'Q4':'Q4 Q1/2015'
                       }, inplace=True)

In [465]:
Q1_2015 = Q1_2015.reset_index()
Q1_2015 = Q1_2015.drop(['index'],axis=1)

Q1 2015 has 35 entries

Slice Q4 2014

In [468]:
Q4_2014 = base_df.ix[base_df['Quarter']=="Q4 2014"]
Q4_2014 = Q4_2014.drop(['ROE','Net Charge-off','Quarter','Bank ID'], axis=1)

Q4_2014.rename(columns={'ROA': 'ROA Q4/2014', 
                        'Loan Loss Provision %': 'Loan Loss Provision % Q4/2014', 
    'Dummy_Loan Loss Provision %': 'Dummy_Loan Loss Provision % Q4/2014',
    'Net Charge Off-to-Loans':'Net Charge Off-to-Loans Q4/2014',
    'Dummy_Net Charge Off-to-Loans': 'Dummy_Net Charge Off-to-Loans Q4/2014',
    'Liquidity': 'Liquidity Q4/2014',
    'Dummy_Liquidity': 'Dummy_Liquidity Q4/2014',
    'Bank Size': 'Bank Size Q4/2014' ,
             'Loan-to-Assets': 'Loan-to-Assets Q4/2014',
             'Leverage': 'Leverage Q4/2014' ,
             'Fixed Assets-to-Assets':'Fixed Assets-to-Assets Q4/2014' ,
'Total derivative contracts held for trading. Equity exchange contracts': 'OBS_Equity Q4/2014',
                        'Dummy_OBS_Equity': 'Dummy_OBS_Equity Q4/2014',
'Total derivative contracts held for trading. Commodity and other contracts': 'OBS_Commodity&Other Q4/2014',
            'Dummy_OBS_Commodity&Other': 'Dummy_OBS_Commodity&Other Q4/2014',
'Total derivative contracts            held for trading. Interest rate contracts':'OBS_Interest Q4/2014',
        'Dummy_OBS_Interest':'dummy_OBS_Interest Q4/2014',
'Total derivative contracts held for trading. Foreign exchange contracts': 'OBS_FX Q4/2014',
        'Dummy_OBS_FX': 'Dummy_OBS_FX Q4/2014',
'Current credit exposure across all off-balance sheet derivatives            contract covered by \
the risk based capital standards':'OBS Credit Exposure Q4/2014',
             'Dummy_OBS_Credit_exposure': 'Dummy_OBS_Credit_exposure Q4/2014',
             'Real GDP ($M)': 'Real GDP ($M) Q4/2014',
            'Inflation (CPI, base year 2010 =100)':'Inflation Q4/2014',
            'Slope of yeild curve': 'Slope of yeild curve Q4/2014',
             'Q2' :'Q2 Q4/2014', 
            'Q3': 'Q3 Q4/2014', 
            'Q4':'Q4 Q4/2014'
                       }, inplace=True)

In [469]:
Q4_2014 = Q4_2014.reset_index()
Q4_2014 = Q4_2014.drop(['index'],axis=1)

Q4 2014 has 90 entries

In [483]:
Q3_2014 = base_df.ix[base_df['Quarter']=="Q3 2014"]
Q3_2014 = Q3_2014.drop(['ROE','Net Charge-off','Quarter','Bank ID'], axis=1)

Q3_2014.rename(columns={'ROA': 'ROA Q3/2014', 
                        'Loan Loss Provision %': 'Loan Loss Provision % Q3/2014', 
    'Dummy_Loan Loss Provision %': 'Dummy_Loan Loss Provision % Q3/2014',
    'Net Charge Off-to-Loans':'Net Charge Off-to-Loans Q3/2014',
    'Dummy_Net Charge Off-to-Loans': 'Dummy_Net Charge Off-to-Loans Q3/2014',
    'Liquidity': 'Liquidity Q3/2014',
    'Dummy_Liquidity': 'Dummy_Liquidity Q3/2014',
    'Bank Size': 'Bank Size Q3/2014' ,
             'Loan-to-Assets': 'Loan-to-Assets Q3/2014',
             'Leverage': 'Leverage Q3/2014' ,
             'Fixed Assets-to-Assets':'Fixed Assets-to-Assets Q3/2014' ,
'Total derivative contracts held for trading. Equity exchange contracts': 'OBS_Equity Q3/2014',
                        'Dummy_OBS_Equity': 'Dummy_OBS_Equity Q3/2014',
'Total derivative contracts held for trading. Commodity and other contracts': 'OBS_Commodity&Other Q3/2014',
            'Dummy_OBS_Commodity&Other': 'Dummy_OBS_Commodity&Other Q3/2014',
'Total derivative contracts            held for trading. Interest rate contracts':'OBS_Interest Q3/2014',
        'Dummy_OBS_Interest':'dummy_OBS_Interest Q3/2014',
'Total derivative contracts held for trading. Foreign exchange contracts': 'OBS_FX Q3/2014',
        'Dummy_OBS_FX': 'Dummy_OBS_FX Q3/2014',
'Current credit exposure across all off-balance sheet derivatives            contract covered by \
the risk based capital standards':'OBS Credit Exposure Q3/2014',
             'Dummy_OBS_Credit_exposure': 'Dummy_OBS_Credit_exposure Q3/2014',
             'Real GDP ($M)': 'Real GDP ($M) Q3/2014',
            'Inflation (CPI, base year 2010 =100)':'Inflation Q3/2014',
            'Slope of yeild curve': 'Slope of yeild curve Q3/2014',
             'Q2' :'Q2 Q3/2014', 
            'Q3': 'Q3 Q3/2014', 
            'Q4':'Q4 Q3/2014'
                       }, inplace=True)

In [484]:
Q3_2014 = Q3_2014.reset_index()
Q3_2014 = Q3_2014.drop(['index'],axis=1)

* Q3_2015 has 31 entries
* Q2_2015 has 31 entries
* Q1_2015 has 35 entries
* Q4_2014 has 90 entries
* Q3_2014 has 45 entries

In [597]:
reg_df = pd.concat([Q3_2015, Q2_2015, Q1_2015, Q4_2014, Q3_2014], axis=1)

In [598]:
#reg_df = Q3_2014.join(Q4_2014)
#reg_df = reg_df.join(Q1_2015)
#reg_df = reg_df.join(Q2_2015)

In [599]:
reg_df.tail()


,ROA Q3/2015,ROA Q2/2015,Loan Loss Provision % Q2/2015,Dummy_Loan Loss Provision % Q2/2015,Net Charge Off-to-Loans Q2/2015,Dummy_Net Charge Off-to-Loans Q2/2015,Liquidity Q2/2015,Dummy_Liquidity Q2/2015,Bank Size Q2/2015,Loan-to-Assets Q2/2015,...,OBS_FX Q3/2014,Dummy_OBS_FX Q3/2014,OBS Credit Exposure Q3/2014,Dummy_OBS_Credit_exposure Q3/2014,Real GDP ($M) Q3/2014,Inflation Q3/2014,Slope of yeild curve Q3/2014,Q2 Q3/2014,Q3 Q3/2014,Q4 Q3/2014
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [607]:
reg_df = reg_df.dropna()
reg_df.to_csv('reg_df.csv')

In [609]:
df1 = df.ix[:,0:2]
X1 = reg_df.ix[:,1:]


In [612]:
from pandas.stats.plm import PanelOLS
X = reg_df.ix[:,1:2]
#X = reg_df['ROA Q2/2015']
Y = reg_df['ROA Q3/2015']
reg  = PanelOLS(Y, X)
reg

NotImplementedError: Only 2-level MultiIndex are supported.